In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
tr_path = "https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv"
ts_path = "https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv"

train_df = pd.read_csv(tr_path)
test_df = pd.read_csv(ts_path)

In [3]:
scale_factor = 1000.0

train_df["median_house_value"] /= scale_factor 

test_df["median_house_value"] /= scale_factor

> ### Functions for building and training our model

In [4]:
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
                
        self.linear = nn.Linear(in_features=1, out_features=1)

    def forward(self, x):
        return self.linear(x)
    
def build_model(my_learning_rate):
    model = LinearRegression()
    loss_fn = nn.MSELoss()
    
    optimizer = optim.RMSprop(model.parameters(), lr=my_learning_rate)
    return model, loss_fn, optimizer

def train_model(model, df, loss_fn, optimizer, feature, label, epochs,
                batch_size, validation_split=0.2):
    epoch_count = []
    train_loss_values = []
    val_loss_values = []
    
    feature_tensor = torch.tensor(df[feature].values,
                                  dtype=torch.float32).unsqueeze(dim=1)
    
    label_tensor = torch.tensor(df[label].values,
                                dtype=torch.float32).unsqueeze(dim=1)
    
    dataset = torch.utils.data.TensorDataset(feature_tensor, label_tensor)
    
    total_samples = len(df)
    train_size = int((1 - validation_split) * total_samples)
    val_size = total_samples - train_size
    
    train_dataset, val_dataset = torch.utils.data.random_split(dataset,
                                                               [train_size, val_size])
    
    
    train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                   batch_size=batch_size, shuffle=True)
    
    val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                                 batch_size=batch_size, shuffle=False)
    
    for epoch in range(epochs):
        model.train()
        for batch_feature, batch_label in train_dataloader:
            train_preds = model(batch_feature)
            train_loss = loss_fn(train_preds, batch_label)
            optimizer.zero_grad()   
            train_loss.backward()   
            optimizer.step()
        epoch_count.append(epoch)
        train_loss_values.append(train_loss.item())
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_feature, batch_label in val_dataloader:
                val_preds = model(batch_feature)
                val_loss  = loss_fn(val_preds, batch_label).item()
        val_loss_values.append(val_loss)
        
        print(f"Epoch: {epoch} | Train Loss: {train_loss.item()} | Validation Loss: {val_loss}")
    
    trained_weight = model.linear.weight.item()
    trained_bias = model.linear.bias.item()
    
    return trained_weight, trained_bias, epoch_count, train_loss_values, val_loss_values

> #### Plotting functions

In [78]:
def plot_the_loss_curve(epochs, tr_error, val_error):
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Root Mean Squared Error")
    plt.plot(epochs, tr_error, label="Training Loss")
    plt.plot(epochs, val_error, label="Validation Loss")
    plt.legend()
    plt.show()

+ In the code below, the `validation_split` variable specifies the proportion of the original training set that will serve as the validation set<br><br>
+ The original training set contains 17,000 examples. Therefore, a validation_split of 0.2 means that:
    + 17,000 * 0.2 ~= 3,400 examples will become the validation set
    + 17,000 * 0.8 ~= 13,600 examples will become the new training set
<br><br>
+ If the data in the training set is similar to the data in the validation set, then the two loss curves and the final loss values should be almost identical<br><br>
+ However, we see that the loss curves and final loss values are not almost identical. Hmm, that's odd<br><br><br><br>

+ Let's experiment with two or three different values of `validation_split` and see if it fixes the problem?<br><br>

In [95]:
learning_rate = 0.04
epochs = 30
batch_size = 100

validation_split = 0.2

my_feature = "median_income"
my_label = "median_house_value"

In [96]:
model, loss_fn, optimizer = build_model(learning_rate)

tr_weight, tr_bias, epochs_list, tr_loss_list, val_loss_list = train_model(model,
                                                                           train_df,
                                                                           loss_fn,
                                                                           optimizer,
                                                                           my_feature,
                                                                           my_label,
                                                                           epochs,
                                                                           batch_size)

Epoch: 0 | Train Loss: 36330.2890625 | Validation Loss: 40109.421875
Epoch: 1 | Train Loss: 28836.259765625 | Validation Loss: 31163.341796875
Epoch: 2 | Train Loss: 22677.369140625 | Validation Loss: 24245.599609375
Epoch: 3 | Train Loss: 18307.63671875 | Validation Loss: 18751.52734375
Epoch: 4 | Train Loss: 16162.33203125 | Validation Loss: 14511.4541015625
Epoch: 5 | Train Loss: 8995.162109375 | Validation Loss: 11489.7138671875
Epoch: 6 | Train Loss: 8012.72265625 | Validation Loss: 9519.931640625
Epoch: 7 | Train Loss: 6278.64208984375 | Validation Loss: 8467.0
Epoch: 8 | Train Loss: 5786.80859375 | Validation Loss: 8020.4794921875
Epoch: 9 | Train Loss: 6116.439453125 | Validation Loss: 7899.5283203125
Epoch: 10 | Train Loss: 6495.21630859375 | Validation Loss: 7875.998046875
Epoch: 11 | Train Loss: 8412.1884765625 | Validation Loss: 7871.29736328125
Epoch: 12 | Train Loss: 5049.42236328125 | Validation Loss: 7875.58740234375
Epoch: 13 | Train Loss: 8243.603515625 | Validation L

In [97]:
plot_the_loss_curve(epochs_list, tr_loss_list, val_loss_list)